In [30]:
import findspark
findspark.init('/home/ubuntu/spark-2.4.6-bin-hadoop2.7')
from pyspark.sql import SparkSession
import pandas as pd
import json

# Convert file to Parquet

In [2]:
spark = SparkSession\
        .builder\
        .appName('Convert file to Parquet')\
        .getOrCreate()

In [3]:
csvFile = spark.read.csv("data/input/users/load.csv",header=True)

In [4]:
csvFile.toPandas()

,id,name,email,phone,address,age,create_date,update_date
0,1,david.lynch@cognitivo.ai,David Lynch,(11) 99999-9997,"Mulholland Drive, Los Angeles, CA, US",72,2018-03-03 18:47:01.954752,2018-03-03 18:47:01.954752
1,1,david.lynch@cognitivo.ai,David Lynch,(11) 99999-9998,"Mulholland Drive, Los Angeles, CA, US",72,2018-03-03 18:47:01.954752,2018-04-14 17:09:48.558151
2,2,sherlock.holmes@cognitivo.ai,Sherlock Holmes,(11) 94815-1623,"221B Baker Street, London, UK",34,2018-04-21 20:21:24.364752,2018-04-21 20:21:24.364752
3,3,spongebob.squarepants@cognitivo.ai,Spongebob Squarepants,(11) 91234-5678,"124 Conch Street, Bikini Bottom, Pacific Ocean",13,2018-05-19 04:07:06.854752,2018-05-19 04:07:06.854752
4,1,david.lynch@cognitivo.ai,David Lynch,(11) 99999-9999,"Mulholland Drive, Los Angeles, CA, US",72,2018-03-03 18:47:01.954752,2018-05-23 10:13:59.594752
5,3,spongebob.squarepants@cognitivo.ai,Spongebob Squarepants,(11) 98765-4321,"122 Conch Street, Bikini Bottom, Pacific Ocean",13,2018-05-19 04:07:06.854752,2018-05-19 05:08:07.964752


In [5]:
csvFile.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- address: string (nullable = true)
 |-- age: string (nullable = true)
 |-- create_date: string (nullable = true)
 |-- update_date: string (nullable = true)



In [8]:
csvFile.write.parquet("data/input/users/load.parquet",mode="overwrite")

In [9]:
parquetFile = spark.read.parquet("data/input/users/load.parquet")

In [11]:
parquetFile.toPandas()

,id,name,email,phone,address,age,create_date,update_date
0,1,david.lynch@cognitivo.ai,David Lynch,(11) 99999-9997,"Mulholland Drive, Los Angeles, CA, US",72,2018-03-03 18:47:01.954752,2018-03-03 18:47:01.954752
1,1,david.lynch@cognitivo.ai,David Lynch,(11) 99999-9998,"Mulholland Drive, Los Angeles, CA, US",72,2018-03-03 18:47:01.954752,2018-04-14 17:09:48.558151
2,2,sherlock.holmes@cognitivo.ai,Sherlock Holmes,(11) 94815-1623,"221B Baker Street, London, UK",34,2018-04-21 20:21:24.364752,2018-04-21 20:21:24.364752
3,3,spongebob.squarepants@cognitivo.ai,Spongebob Squarepants,(11) 91234-5678,"124 Conch Street, Bikini Bottom, Pacific Ocean",13,2018-05-19 04:07:06.854752,2018-05-19 04:07:06.854752
4,1,david.lynch@cognitivo.ai,David Lynch,(11) 99999-9999,"Mulholland Drive, Los Angeles, CA, US",72,2018-03-03 18:47:01.954752,2018-05-23 10:13:59.594752
5,3,spongebob.squarepants@cognitivo.ai,Spongebob Squarepants,(11) 98765-4321,"122 Conch Street, Bikini Bottom, Pacific Ocean",13,2018-05-19 04:07:06.854752,2018-05-19 05:08:07.964752


In [12]:
parquetFile.printSchema()

root
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- address: string (nullable = true)
 |-- age: string (nullable = true)
 |-- create_date: string (nullable = true)
 |-- update_date: string (nullable = true)



# Deduplicar utilizando como referência o update_date

In [13]:
parquetFileDeduplicado = parquetFile.groupby('id').agg({'update_date':'max'}).withColumnRenamed("max(update_date)", "update_date")

In [14]:
parquetFileDeduplicado.toPandas()

,id,update_date
0,3,2018-05-19 05:08:07.964752
1,1,2018-05-23 10:13:59.594752
2,2,2018-04-21 20:21:24.364752


In [17]:
parquetFinal = parquetFile.join(parquetFileDeduplicado,['id','update_date'],"inner")

In [18]:
parquetFinal.toPandas()

,id,update_date,name,email,phone,address,age,create_date
0,2,2018-04-21 20:21:24.364752,sherlock.holmes@cognitivo.ai,Sherlock Holmes,(11) 94815-1623,"221B Baker Street, London, UK",34,2018-04-21 20:21:24.364752
1,1,2018-05-23 10:13:59.594752,david.lynch@cognitivo.ai,David Lynch,(11) 99999-9999,"Mulholland Drive, Los Angeles, CA, US",72,2018-03-03 18:47:01.954752
2,3,2018-05-19 05:08:07.964752,spongebob.squarepants@cognitivo.ai,Spongebob Squarepants,(11) 98765-4321,"122 Conch Street, Bikini Bottom, Pacific Ocean",13,2018-05-19 04:07:06.854752


In [25]:
parquetFinal.write.parquet("data/input/users/loadFinal.parquet",mode="overwrite")

# Mudar tipo da coluna baseado no Json

In [29]:
parquetFinal.printSchema()

root
 |-- id: string (nullable = true)
 |-- update_date: string (nullable = true)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- address: string (nullable = true)
 |-- age: string (nullable = true)
 |-- create_date: string (nullable = true)



In [35]:
jsonFile = open('config/types_mapping.json')
jsonData = json.load(jsonFile)
jsonFile.close()

for (k, v) in jsonData.items():
    print("Key: " + k, "Value: " + str(v))
    parquetFinal = parquetFinal.withColumn(k, parquetFinal[k].cast(v))

Key: age Value: integer
Key: create_date Value: timestamp
Key: update_date Value: timestamp


In [36]:
parquetFinal.printSchema()

root
 |-- id: string (nullable = true)
 |-- update_date: timestamp (nullable = true)
 |-- name: string (nullable = true)
 |-- email: string (nullable = true)
 |-- phone: string (nullable = true)
 |-- address: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- create_date: timestamp (nullable = true)



In [37]:
parquetFinal.toPandas()

,id,update_date,name,email,phone,address,age,create_date
0,2,2018-04-21 20:21:24.364752,sherlock.holmes@cognitivo.ai,Sherlock Holmes,(11) 94815-1623,"221B Baker Street, London, UK",34,2018-04-21 20:21:24.364752
1,1,2018-05-23 10:13:59.594752,david.lynch@cognitivo.ai,David Lynch,(11) 99999-9999,"Mulholland Drive, Los Angeles, CA, US",72,2018-03-03 18:47:01.954752
2,3,2018-05-19 05:08:07.964752,spongebob.squarepants@cognitivo.ai,Spongebob Squarepants,(11) 98765-4321,"122 Conch Street, Bikini Bottom, Pacific Ocean",13,2018-05-19 04:07:06.854752


In [38]:
parquetFinal.write.parquet("data/input/users/loadFinalDataTypesFormatted.parquet",mode="overwrite")